In [1]:
import tensorflow as tf
import numpy as np 

In [5]:
char_rdic = ['a', 'p', 'l', 'e', 'i', 'n'] # id -> char
char_dic = {w : i for i, w in enumerate(char_rdic)} # char -> id
print (char_dic)
print (len(char_dic))

{'a': 0, 'e': 3, 'i': 4, 'l': 2, 'n': 5, 'p': 1}
6


In [3]:
ground_truth = [char_dic[c] for c in 'pineapple']
print (ground_truth)

[1, 4, 5, 3, 0, 1, 1, 2, 3]


In [4]:
x_data = np.array([[0,1,0,0,0,0], # p
                   [0,0,0,0,1,0], # i
                   [0,0,0,0,0,1], # n
                   [0,0,0,1,0,0], # e
                   [1,0,0,0,0,0]], # a
                 dtype = 'f')

In [7]:
x_data = tf.one_hot(ground_truth[:-1], len(char_dic), 1.0, 0.0, -1)
print(x_data)

Tensor("one_hot:0", shape=(8, 6), dtype=float32)


In [8]:
# Configuration
rnn_size = len(char_dic) # 4
batch_size = 1
output_size = 6

print(rnn_size)

6


In [9]:
# RNN Model
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = rnn_size,
                                       input_size = None, # deprecated at tensorflow 0.9
                                       #activation = tanh,
                                       )
print(rnn_cell)

In [10]:
initial_state = rnn_cell.zero_state(batch_size, tf.float32)
print(initial_state)

Tensor("zeros:0", shape=(1, 6), dtype=float32)


In [11]:
initial_state_1 = tf.zeros([batch_size, rnn_cell.state_size]) #  위 코드와 같은 결과
print(initial_state_1)

Tensor("zeros_1:0", shape=(1, 6), dtype=float32)


In [12]:
x_split = tf.split(0, len(char_dic), x_data) # 가로축으로 4개로 split
print(x_split)

ValueError: Number of ways to split should evenly divide the split dimension but got split_dim 0 (size = 8) and num_split 6

In [34]:
outputs, state = tf.nn.rnn(cell = rnn_cell, inputs = x_split, initial_state = initial_state)

ValueError: Variable RNN/BasicRNNCell/Linear/Matrix already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-19-542d964eb458>", line 1, in <module>
    outputs, state = tf.nn.rnn(cell = rnn_cell, inputs = x_split, initial_state = initial_state)
  File "/s/anaconda/users/jkim101/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/s/anaconda/users/jkim101/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


In [29]:
print (outputs)

[<tf.Tensor 'RNN/BasicRNNCell/Tanh:0' shape=(1, 6) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_1/Tanh:0' shape=(1, 6) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_2/Tanh:0' shape=(1, 6) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_3/Tanh:0' shape=(1, 6) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_4/Tanh:0' shape=(1, 6) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_5/Tanh:0' shape=(1, 6) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_6/Tanh:0' shape=(1, 6) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_7/Tanh:0' shape=(1, 6) dtype=float32>]


In [30]:
print (state)

Tensor("RNN/BasicRNNCell_7/Tanh:0", shape=(1, 6), dtype=float32)


In [31]:
# predict
logits = tf.reshape(tf.concat(1, outputs), # shape = 1 x 16
                    [-1, rnn_size])        # shape = 4 x 4
logits.get_shape()
"""
[[logit from 1st output],
[logit from 2nd output],
[logit from 3rd output],
[logit from 4th output]]
"""

'\n[[logit from 1st output],\n[logit from 2nd output],\n[logit from 3rd output],\n[logit from 4th output]]\n'

In [32]:
# actual
targets = tf.reshape(ground_truth[1:], [-1]) # a shape of [-1] flattens into 1-D
targets.get_shape()

TensorShape([Dimension(8)])

In [33]:
weights = tf.ones([len(char_dic) * batch_size])

In [25]:
loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], [weights])
cost = tf.reduce_sum(loss) / batch_size
train_op = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(cost)

ValueError: Incompatible shapes for broadcasting: (8,) and (6,)

In [18]:
# Launch the graph in a session
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for i in range(100):
        sess.run(train_op)
        result = sess.run(tf.argmax(logits, 1))
        print(result, [char_rdic[t] for t in result])

(array([0, 1, 1, 1]), ['a', 'p', 'p', 'p'])
(array([2, 1, 1, 1]), ['l', 'p', 'p', 'p'])
(array([2, 1, 1, 1]), ['l', 'p', 'p', 'p'])
(array([2, 1, 1, 1]), ['l', 'p', 'p', 'p'])
(array([2, 1, 1, 1]), ['l', 'p', 'p', 'p'])
(array([2, 1, 1, 1]), ['l', 'p', 'p', 'p'])
(array([2, 1, 1, 1]), ['l', 'p', 'p', 'p'])
(array([2, 1, 1, 3]), ['l', 'p', 'p', 'e'])
(array([3, 1, 1, 3]), ['e', 'p', 'p', 'e'])
(array([3, 1, 1, 3]), ['e', 'p', 'p', 'e'])
(array([3, 1, 1, 3]), ['e', 'p', 'p', 'e'])
(array([3, 1, 1, 3]), ['e', 'p', 'p', 'e'])
(array([3, 1, 1, 3]), ['e', 'p', 'p', 'e'])
(array([3, 1, 1, 3]), ['e', 'p', 'p', 'e'])
(array([3, 1, 1, 3]), ['e', 'p', 'p', 'e'])
(array([3, 1, 1, 3]), ['e', 'p', 'p', 'e'])
(array([3, 1, 1, 3]), ['e', 'p', 'p', 'e'])
(array([1, 1, 1, 3]), ['p', 'p', 'p', 'e'])
(array([1, 1, 1, 3]), ['p', 'p', 'p', 'e'])
(array([1, 1, 1, 3]), ['p', 'p', 'p', 'e'])
(array([1, 1, 1, 3]), ['p', 'p', 'p', 'e'])
(array([1, 1, 1, 3]), ['p', 'p', 'p', 'e'])
(array([1, 1, 1, 3]), ['p', 'p',